# ✈️ Airline Sentiment Analysis

**Autor:** Emilly Cavalcante  
**Data:** Novembro 2025  
**Objetivo:** Analisar sentimentos em avaliações de companhias aéreas usando NLP.

## Sobre o Projeto

Este projeto busca realizar a análise de sentimentos em avaliações de passageiros de companhias aéreas usando um modelo transformer pré-treinado (DistilBERT). O principal objetivo é identificar padrões de satisfação e insatisfação no setor de aviação.

**Dataset:** Airline Reviews (Kaggle)  
**Modelo:** lxyuan/distilbert-base-multilingual-cased-sentiments-student  
**Amostra:** 5.000 avaliações de 23.171 totais

---
## 1. Importação de bibliotecas e carregamento do modelo

Importando as bibliotecas necessárias para manipulação de dados, visualização e processamento de linguagem natural. Além disso, fazendo uso do modelo **DistilBERT multilíngue** fine-tuned para análise de sentimentos.

**Características do modelo:**
- Baseado em BERT, mas mais leve e rápido
- Suporta múltiplos idiomas
- Classifica em: positive, negative, neutral
- Retorna score de confiança (0-1)

In [ ]:
import pandas as pd
import plotly.express as px
from transformers import pipeline

model_path = 'lxyuan/distilbert-base-multilingual-cased-sentiments-student'
classifier = pipeline('sentiment-analysis', model=model_path, tokenizer=model_path, truncation=True, max_length=512)

---
## 2. Carregamento e exploração dos dados

Nessa seção é realizado o carregamento do dataset de avaliações de companhias aéreas e também é feita uma exploração inicial de sua estrutura.

In [ ]:
df = pd.read_csv('airline_reviews.csv', engine='python', on_bad_lines='skip')

print("Informações do dataset:")
df.info()

In [ ]:
# visualização das 10 primeiras linhas
df.head(10)

In [ ]:
# visualização das 10 últimas linhas
df.tail(10)

---
## 3. Pré-processamento dos dados

Remoção de colunas desnecessárias e criação de uma amostra representativa para análise.

In [ ]:
df = df.drop(columns=['Unnamed: 0', 'Verified', 'Aircraft', 'Date Flown', 'Seat Comfort', 'Cabin Staff Service', 'Food & Beverages',
                      'Ground Service', 'Inflight Entertainment', 'Wifi & Connectivity', 'Value For Money'])

print("Dataset após remoção de colunas desnecessárias:")
df.head()

In [ ]:
# criação de uma amostra aleatória de 5000 linhas
sample_df = df.sample(n=5000, random_state=42, ignore_index=True)

---
## 4. Aplicação do modelo de sentimentos

### Teste em amostras

Primeiro, testando o modelo em algumas avaliações para verificar o funcionamento:

In [ ]:
print("Teste do modelo em 5 avaliações:\n")

for i in range(5):
  print(classifier(sample_df.loc[i, 'Review']))

### Processamento completo

Aplicando o modelo em todas as 5.000 avaliações da amostra:

In [ ]:
label = []
score = []

for i in range(len(sample_df)):
  result = classifier(sample_df.loc[i, 'Review'])

  label.append(result[0]['label'])
  score.append(result[0]['score'])

  if i % 500 == 0:
    print(f'Processed {i} reviews')

In [ ]:
# adicionando resultados da análise ao dataframe
sample_df['Sentiment'] = label
sample_df['Sentiment Score'] = score

In [ ]:
# salvando o novo dataframe como arquivo csv
sample_df.to_csv('results_with_sentiment.csv', index=False)

---
## 5. Análise 1: Distribuição de sentimentos

Nessa primeira análise é verificada a proporção de avaliações positivas, negativas e neutras.

In [ ]:
# total de avaliações positivas, negativas e neutras
sample_df['Sentiment'].value_counts()

In [ ]:
fig1= px.pie(sample_df, names='Sentiment', title='Distribuição de sentimentos nas avaliações')
fig1.show()

### Insights

- **63.7%** das avaliações são **negativas**, ou seja, a maioria dos passageiros expressa insatisfação com as companhias aéreas.
- **31.4%** são **positivas**, o que significa que apenas cerca de 1/3 das experiências são satisfatórias.
- **4.9%** são **neutras**, sentimentos neutros são raros o que indica que experiências de voo costumam gerar opiniões polarizadas.

---
## 6. Análise 2: Relação entre nota e sentimento detectado

Investigando se existe correlação entre a nota numérica atribuída pelo usuário (1-9) e o sentimento identificado pelo modelo.

In [ ]:
# criação de tabela cruzada
sentiment_rating_tab = pd.crosstab(sample_df['Sentiment'], sample_df['Overall_Rating'])
sentiment_rating_tab = sentiment_rating_tab.drop(columns=['n'])

sentiment_rating_tab

In [ ]:
fig2 = px.imshow(
    sentiment_rating_tab,
    text_auto=True,
    labels=dict(x="Nota Geral", y="Sentimento Detectado"),
    color_continuous_scale='Blues'
)

fig2.update_layout(title="Relação: Nota vs Sentimento Detectado")
fig2.show()

### Insights

É possível observar uma forte correlação entre a nota atribuída pelo usuário e o sentimento detectado pelo modelo:

- **Notas 1-2:** Quase exclusivamente negativas (1.896 de 2.467 com nota 1 são negativas).
- **Notas 3-5:** Predominância de negativos, mas com alguns positivos aparecendo.
- **Notas 6-7:** Maior diversidade, transição entre negativo e positivo.
- **Notas 8-9:** Predominantemente positivas (327 avaliações com nota 9 são positivas).

A consistência entre a nota numérica e o sentimento detectado indica que o modelo está classificando corretamente.

**Observação:** Mesmo com notas baixas (1-3), há algumas avaliações classificadas como neutras ou positivas que, possivelmente, são casos nos quais o usuário:
- Deu nota baixa por um aspecto específico, mas o texto não expressa tanto negativismo.
- Escreveu de forma objetiva/neutra.

---
## 7. Análise 3: Sentimento médio por campanhia aérea

In [ ]:
# total de companhias únicas
sample_df['Airline Name'].nunique()

In [ ]:
# top 15 companhias com mais avaliações
top15_companies = sample_df['Airline Name'].value_counts().head(15)
top15_companies

In [ ]:
# dataset apenas com dados das 15 companhias mais avaliadas
top_companies = sample_df[sample_df['Airline Name'].isin(top15_companies.index)]
top_companies

In [ ]:
sentiments_per_company = top_companies.groupby('Airline Name')['Sentiment'].value_counts()
sentiments_per_company

In [ ]:
results = []

for company in top_companies['Airline Name'].unique():
  company_df = top_companies[top_companies['Airline Name'] == company]

  total = len(company_df)

  positive = len(company_df[company_df['Sentiment'] == 'positive'])
  negative = len(company_df[company_df['Sentiment'] == 'negative'])
  neutral = len(company_df[company_df['Sentiment'] == 'neutral'])

  positive_percentage = (positive / total) * 100
  negative_percentage = (negative / total) * 100
  neutral_percentage = (neutral / total) * 100

  results.append({
      'Airline Name': company,
      'Total': total,
      'Positive': positive,
      'Negative': negative,
      'Neutral': neutral,
      'Positive %': positive_percentage,
      'Negative %': negative_percentage,
      'Neutral %': neutral_percentage
  })

results_df = pd.DataFrame(results)
results_df.sort_values(by='Positive %', ascending=False)

In [ ]:
fig3 = px.bar(
    results_df,
    x='Positive %',
    y='Airline Name',
    color='Airline Name',
    orientation='h',
    title='Sentimento positivo médio por companhia aérea')

fig3.update_layout(yaxis={'categoryorder':'total ascending'})

fig3.show()

### Insights

* Apenas **3 das 15 companhias** (20%) têm maioria de avaliações positivas.
* **12 companhias** (80% do top 15) possuem mais de 55% de avaliações negativas.
* A **Aer Lingus** é a pior avaliada, com cerca de 92% de avaliações negativas e apenas 3% de avaliações positivas.
* A **Thai Smile Airways** é um destaque positivo com 78.6% de avaliações positivas, sendo a única acima de 70%.

É possível observar um padrão geral no setor que é o de insatisfação.

**Observação:** A diferença entre a melhor (78.6%) e a pior (3.7%) é de 75 pontos percentuais.


## 8. Análise 4: Evolução do sentimento ao longo dos anos

In [ ]:
suffixes = ['th', 'st', 'nd', 'rd']
sample_df['Review Date'] = sample_df['Review Date'].str.replace('|'.join(suffixes), '', regex=True)
sample_df['Review Date'] = sample_df['Review Date'].str.replace('Augu', 'August')

sample_df['Review Date'].head(10)

In [ ]:
sample_df['Review Date'] = pd.to_datetime(sample_df['Review Date'], format='%d %B %Y')
sample_df['Review Date'].head(10)

In [ ]:
print(f'Data mais antiga: {sample_df['Review Date'].min()}')
print(f'Data mais recente: {sample_df['Review Date'].max()}')

In [ ]:
sample_df['Review Date'].dt.year.value_counts()

In [ ]:
year_sample = sample_df[sample_df['Review Date'] >= '2015-01-01']
year_sample

In [ ]:
sentiment_per_year = []

for year in year_sample['Review Date'].dt.year.unique():
  reviews_in_year = year_sample[year_sample['Review Date'].dt.year == year]

  total_reviews = len(reviews_in_year)

  positive_reviews = len(reviews_in_year[reviews_in_year['Sentiment'] == 'positive'])
  neutral_reviews = len(reviews_in_year[reviews_in_year['Sentiment'] == 'neutral'])
  negative_reviews = len(reviews_in_year[reviews_in_year['Sentiment'] == 'negative'])

  positive_percentage = (positive_reviews / total_reviews) * 100
  neutral_percentage = (neutral_reviews / total_reviews) * 100
  negative_percentage = (negative_reviews / total_reviews) * 100

  sentiment_per_year.append({
      'Year': year,
      'Total Reviews': total_reviews,
      'Positive': positive_reviews,
      'Neutral': neutral_reviews,
      'Negative': negative,
      'Positive %': positive_percentage,
      'Neutral %': neutral_percentage,
      'Negative %': negative_percentage
  })

sentiment_per_year_df = pd.DataFrame(sentiment_per_year)
sentiment_per_year_df = sentiment_per_year_df.sort_values(by='Year')
sentiment_per_year_df

In [ ]:
df_melted = sentiment_per_year_df.melt(
    id_vars=['Year'],
    value_vars=['Positive %', 'Neutral %', 'Negative %'],
    var_name='Sentiment',
    value_name='Percentual'
)

fig4 = px.line(
    df_melted,
    x='Year',
    y='Percentual',
    color='Sentiment',
    markers=True,
    title='Evolução do Sentimento ao Longo dos Anos'
)

fig4.show()

### Insights

* A satisfação com companhias aéreas caiu de forma consistente de **2015 a 2023**.
* **2016** foi o pico com 45.4% de avaliações positivas.
* **2023** é o pior ano, tendo apenas 20.6% positivas o que representa uma **queda de 54%** em relação a 2016.